In [1]:
from datetime import datetime

from pymongo import MongoClient
from pprint import pprint

In [2]:
client = MongoClient(port=27017)

In [3]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [4]:
students_db = client.students

In [5]:
students_db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'students')

In [6]:
student_scores = students_db.studentscores

In [7]:
student_scores

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'students'), 'studentscores')

In [8]:
data = [
  {
    "user": "Krish",
    "subject": "Database",
    "score": 80
  },
  {
    "user": "Amit",
    "subject": "JavaScript",
    "score": 90
  },
  {
    "user": "Amit",
    "title": "Database",
    "score": 85
  },
  {
    "user": "Krish",
    "title": "JavaScript",
    "score": 75
  },
  {
    "user": "Amit",
    "title": "Data Science",
    "score": 60
  },
  {
    "user": "Krish",
    "title": "Data Science",
    "score": 95
  }
]
student_scores.insert_many(data) 

InsertManyResult([ObjectId('681f0caff1fea12ff47d1c76'), ObjectId('681f0caff1fea12ff47d1c77'), ObjectId('681f0caff1fea12ff47d1c78'), ObjectId('681f0caff1fea12ff47d1c79'), ObjectId('681f0caff1fea12ff47d1c7a'), ObjectId('681f0caff1fea12ff47d1c7b')], acknowledged=True)

## Group By

To find how many subjects each student has taken up.

In [9]:
total_subjects_by_student = student_scores.aggregate(
    [{
        "$group": {
            "_id": "$user",
            "Total Subjects": {
                "$sum": 1
            }
        }
    }]
)

In [10]:
for record in total_subjects_by_student:
    pprint(record)

{'Total Subjects': 3, '_id': 'Amit'}
{'Total Subjects': 3, '_id': 'Krish'}


To find the total score of students

In [11]:
total_marks_by_student = student_scores.aggregate(
    [{
        "$group": {
            "_id": "$user",
            "Total score": {"$sum": "$score"}
        }
    }]
)

In [12]:
for record in total_marks_by_student:
    pprint(record)

{'Total score': 235, '_id': 'Amit'}
{'Total score': 250, '_id': 'Krish'}


To find the average sccore of students

In [13]:
average_score_by_student = student_scores.aggregate(
    [{
        "$group": {
            "_id": "$user",
            "Average score": {"$avg": "$score"}
        }
    }]
)

In [14]:
for record in average_score_by_student:
    pprint(record)

{'Average score': 78.33333333333333, '_id': 'Amit'}
{'Average score': 83.33333333333333, '_id': 'Krish'}


In [15]:
data = [
    { "_id" : 1, "item" : "abc", "price" : 10, "quantity" : 2, "date" : datetime.utcnow()},
    { "_id" : 2, "item" : "jkl", "price" : 20, "quantity" : 1, "date" : datetime.utcnow() },
    { "_id" : 3, "item" : "xyz", "price" : 5, "quantity" : 5, "date" : datetime.utcnow() },
    { "_id" : 4, "item" : "abc", "price" : 10, "quantity" : 10, "date" : datetime.utcnow() },
    { "_id" : 5, "item" : "xyz", "price" : 5, "quantity" : 10, "date" : datetime.utcnow() }
]

In [16]:
data

[{'_id': 1,
  'item': 'abc',
  'price': 10,
  'quantity': 2,
  'date': datetime.datetime(2025, 5, 10, 8, 22, 8, 76139)},
 {'_id': 2,
  'item': 'jkl',
  'price': 20,
  'quantity': 1,
  'date': datetime.datetime(2025, 5, 10, 8, 22, 8, 76139)},
 {'_id': 3,
  'item': 'xyz',
  'price': 5,
  'quantity': 5,
  'date': datetime.datetime(2025, 5, 10, 8, 22, 8, 76139)},
 {'_id': 4,
  'item': 'abc',
  'price': 10,
  'quantity': 10,
  'date': datetime.datetime(2025, 5, 10, 8, 22, 8, 76139)},
 {'_id': 5,
  'item': 'xyz',
  'price': 5,
  'quantity': 10,
  'date': datetime.datetime(2025, 5, 10, 8, 22, 8, 76139)}]

In [17]:
store_db = client.store

In [18]:
stores_collection = store_db.stores

In [19]:
stores_collection.insert_many(data)

InsertManyResult([1, 2, 3, 4, 5], acknowledged=True)

To calculate total average price

In [20]:
average_quantity_and_price = stores_collection.aggregate(
    [{
        "$group": {
            "_id": "$item",
            "Total average Price": {"$avg": {"$multiply": ['$price', '$quantity']}},
            "Average quantity": {"$avg": "$quantity"}
        }
    }]
)

In [21]:
for record in average_quantity_and_price:
    pprint(record)

{'Average quantity': 1.0, 'Total average Price': 20.0, '_id': 'jkl'}
{'Average quantity': 7.5, 'Total average Price': 37.5, '_id': 'xyz'}
{'Average quantity': 6.0, 'Total average Price': 60.0, '_id': 'abc'}
